In [1]:
import pandas as pd
import numpy as np
import os
import glob

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
from helpers import *

# Data prep

Läs in fil

In [3]:
df = pd.read_csv('../indata/ODB_TSV/ODB-utbildningsnivå, ålder, tid och kön 2015-2017.txt', sep='\t', header=1)

Specificera indexering

In [4]:
df = df.set_index(['område', 'utbildningsnivå'])

Vänd ner samtliga kolumner i en kolumn

In [5]:
df = pd.DataFrame(df.stack())

Lyft ut index till kolumner igen

In [6]:
df = df.reset_index()

Sätt beskrivande kolumnnamn

In [7]:
df = df.rename(columns={'level_2': 'ageyeargender', 0: 'value'})

Splitta områdesnamn till endast baskod

In [8]:
df['område'] = df['område'].str.split(' ', 1, expand = True)
df = df.rename(columns={'område': 'BASKOD2010'})
df['BASKOD2010'] = df['BASKOD2010'].astype(str).astype(int)

Byt om `BASKOD2010` till entity-namnet

In [9]:
df = baskod2010tobasomrade(df)

Splitta ut `ageyeargender` till fyra separata kolumner

In [10]:
df['agegroup'], df['kasta'], df['year'], df['gender'] = df['ageyeargender'].str.split(' ').str

Kasta kolumner som inte längre är relevanta

In [11]:
df = df.drop('kasta', 1)
df = df.drop('ageyeargender', 1)

Specificera ordningen på kolumnerna

In [12]:
df = df[['basomrade','year','utbildningsnivå', 'gender', 'agegroup', 'value']]

Specificera indexering

In [13]:
df = df.set_index(['basomrade'])

Slå samman åldersgrupper till en `25-64` grupp.

In [14]:
df = df[df['agegroup'] != '20-24']

In [15]:
df = df.groupby(['basomrade', 'year', 'utbildningsnivå', 'gender']).sum().reset_index()

Byt namn på `gender` kolumnen för att följa konventionen i övriga notebooks.

In [16]:
df = df.rename(columns={'gender':'Kön'})

Sätt `Kön` till en pandas category

In [17]:
df['Kön'] = df['Kön'].astype('category')
df['Kön'].cat.categories = ['male', 'female']

# Antal

Exportera

In [18]:
concept_key = {
    'Eftergymnasial 3- år': 'higher_education_min_3_years_aged_25_64',
    'Eftergymnasial < 3 år': 'higher_education_max_3_years_aged_25_64',
    'Förgymnasial': 'pre_secondary_school_aged_25_64',
    'Gymnasial': 'secondary_school_aged_25_64'
}

In [70]:
for i in concept_key:
    concept = 'educational_level_{level}'.format(level=concept_key[i])
    level = df[df['utbildningsnivå'] == i]
    byGender(concept, level)
    level = level.groupby(['basomrade', 'year']).sum().reset_index()
#     print(level.head())
    appendNewDatapoints(concept, level)

Saved educational_level_higher_education_min_3_years_aged_25_64_male to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--educational_level_higher_education_min_3_years_aged_25_64_male--by--basomrade--year.csv

Saved educational_level_higher_education_min_3_years_aged_25_64_female to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--educational_level_higher_education_min_3_years_aged_25_64_female--by--basomrade--year.csv

Saved educational_level_higher_education_min_3_years_aged_25_64 to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--educational_level_higher_education_min_3_years_aged_25_64--by--basomrade--year.csv

Saved educational_level_higher_education_max_3_years_aged_25_64_male to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--educational_level_higher_education_max_3_years_aged_25_64_male--by--basomrade--year.csv

Saved educational_level_higher_education_m

# Andel högutbildade

In [60]:
concept = 'post_secondary_education_min_3_years_aged_25_64'

### Andel per kön

In [61]:
bygender = df.groupby(['basomrade', 'year', 'Kön']).sum().add_prefix('total_').reset_index()

In [62]:
bygender = pd.merge(df, bygender)

In [63]:
bygender['rate'] = bygender['value'] / bygender['total_value']

Vi är bara intresserade av andel högutbildade

In [64]:
bygender = bygender[bygender['utbildningsnivå'].isin(['Eftergymnasial 3- år', 'Eftergymnasial < 3 år'])]

Addera andelarna för de två kategorierna högutbildade.

In [65]:
bygender = bygender.groupby(['basomrade', 'year', 'Kön']).sum().reset_index()
bygender = bygender[['basomrade', 'year', 'Kön', 'rate']]
bygender = bygender.rename(columns={'rate': 'value'})

Exportera

In [66]:
byGender(concept, bygender)

Saved post_secondary_education_min_3_years_aged_25_64_male to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--post_secondary_education_min_3_years_aged_25_64_male--by--basomrade--year.csv

Saved post_secondary_education_min_3_years_aged_25_64_female to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--post_secondary_education_min_3_years_aged_25_64_female--by--basomrade--year.csv



### Andel totalt

In [47]:
total = df.groupby(['basomrade', 'year']).sum().add_prefix('total_').reset_index()
rate = df

In [48]:
rate = rate[rate['utbildningsnivå'].isin(['Eftergymnasial 3- år', 'Eftergymnasial < 3 år'])]

In [51]:
rate = rate.groupby(['basomrade', 'year']).sum().reset_index()

In [53]:
rate = pd.merge(total, rate)

In [55]:
rate['rate'] = rate['value'] / rate['total_value']

In [58]:
rate = rate[['basomrade', 'year', 'rate']]
rate = rate.rename(columns={'rate': 'value'})

In [67]:
appendNewDatapoints(concept, rate)

Saved post_secondary_education_min_3_years_aged_25_64 to ../ddf--sodertornsmodellen-output/ddf--sodertornsmodellen--src/ddf--datapoints--post_secondary_education_min_3_years_aged_25_64--by--basomrade--year.csv

